# OpenCL Vector addition Program

## Steps involved in Creating this program:

1. Create a vector of cl::Platform type because I don't know.

2. Get the Platform by using the c++ wrapper:
    cl::Platform::get(&platforms);

3. Create a vector of Devices with type cl::Device because there can be more devices than one.

4. Get the first platform by using the front() function of vector class. auto platform = platforms.front();

5. Get the devices using platform.getDevices(CL_DEVICE_TYPE_GPU, &devices); This will store all the devices in the devices vector.

6. Get the device that you want to use, for me this was the last device as my AMD radeon was the last one and save it in device variable.

7. Get the source of the kernel code into a string variable (src). You can create a file and read it in the program dynamically, for some reason it was not working for me so I hardcoded the string.

8. Now we have to pass this string as a source of the kernel program which we'll be building.

    cl::Program::Sources source(1, std::make_pair(src.c_str(), src.length() + 1));

9. Create a context by passing the device the you want to use.
    cl::Context context(device);

10. Now create the program by passing in sources and context in the constructor.
    cl::Program program(context, sources);

11. Now build the program with .build() of the cl::Program class instance. This can also return a error code if failed, which might be useful for debugging. (I was having trouble with the source so it was throwing an error and this was useful in identifying the probem, which was explained in step 7.

12. Since these steps are going to be the same in most of the program, I've created a header file which does this in a function and returns a cl::Program.

13. Because of step 12 we need to get some variables back from the program instance.
    

    i. auto context = program.getInfo<CL_PROGRAM_CONTEXT>();
    ii. auto devices = context.getInfo<CL_CONTEXT_DEVICES>();
    iii. auto device = devices.back(); // because I want my 2nd GPU to be used.

14. Create variables that need to be passed on the kernel functor.

15. Create the buffers that can be read by the kernel functor.

    i. cl::Buffer x_buffer;
    ii. x_buffer = cl::Buffer(context, begin(x), end(x), true);


16. The output buffer that needs to be read by the host machine needs to be initialised in a different way.

    i. cl::Buffer out_buffer(context, CL_MEM_WRITE_ONLY, sizeof(double) * out.size());

17. Now make the kernel and pass the program and kernel function name

    i. auto vadd = cl::make_kernel<cl::Buffer, cl::Buffer, cl::Buffer>(program, "vadd");

18. Create a Command Queue which is necessary for creating a parallel processing.

    i. cl::CommandQueue queue(context, device);

19. Pass the buffers that we created as functor variables.

    i. vadd(
        cl::EnqueueArgs(10000000)),
        a_buffer,
        b_buffer,
        out_buffer);

20. Wait for the queue to finish using the queue.finish();

21. Copy the output variable back to the host in a vector.

    i. cl::copy(queue, out_buffer, begin(out_variable), end(out_variable));

22. If you want to get the information of the device that was used in the functor, you can get it using getInfo<CL_DEVICE_XYZ>() where it will return the information.

23. Output the variable or store it in a file.

#### Thats all